# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [16]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tubarjal,30.4999,38.2160,25.93,37,35,1.53,SA,1731251026
1,1,edinburgh of the seven seas,-37.0676,-12.3116,15.81,87,100,10.68,SH,1731251027
2,2,viedma,-40.8135,-62.9967,17.01,28,100,2.54,AR,1731250977
3,3,stanley,54.8680,-1.6985,11.82,92,75,3.09,GB,1731251029
4,4,ribeira grande,38.5167,-28.7000,20.22,94,75,7.20,PT,1731251030


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
%%capture --no-display
# Ensure hvplot and geoviews are imported
import hvplot.pandas
import geoviews as gv
gv.extension('bokeh')

# Configure the map plot
city_map = city_data_df.hvplot.points(
    'Lng', 'Lat',  # Longitude and latitude for plotting points
    geo=True,      # Enable geospatial plotting
    size='Humidity',  # Size of the points based on the 'Humidity' column
    color='Humidity',  # Color the points based on the 'Humidity' column
    tiles='OSM',    # Use OpenStreetMap as the base map
    frame_width=800,
    frame_height=600,
    title='City Map with Humidity Levels'
)

# Display the map
city_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] >= 20) & 
    (city_data_df["Max Temp"] <= 30) &
    (city_data_df["Humidity"] < 60) &
    (city_data_df["Cloudiness"] < 20)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
30,30,alamos,27.0167,-108.9333,21.37,36,0,0.76,MX,1731251059
53,53,tamanrasset,22.7850,5.5228,23.95,16,0,2.57,DZ,1731251086
71,71,al ghayzah,16.2079,52.1760,27.48,55,4,4.64,YE,1731251107
92,92,al bawiti,28.3492,28.8659,21.20,34,0,4.00,EG,1731251130
97,97,hoshangabad,22.7500,77.7167,22.93,50,0,1.34,IN,1731251135


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# Use the Pandas copy function to create a DataFrame called hotel_df
hotel_df = ideal_weather_df.copy()

# Select only the relevant columns: city, country, latitude, longitude, and humidity
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column for "Hotel Name"
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df




,City,Country,Lat,Lng,Humidity,Hotel Name
30,alamos,MX,27.0167,-108.9333,36,
53,tamanrasset,DZ,22.7850,5.5228,16,
71,al ghayzah,YE,16.2079,52.1760,55,
92,al bawiti,EG,28.3492,28.8659,34,
97,hoshangabad,IN,22.7500,77.7167,50,
147,parauna,BR,-16.9478,-50.4486,55,
156,al kharijah,EG,25.4514,30.5464,30,
196,khailar,IN,25.3500,78.5333,37,
240,diplo,PK,24.4667,69.5840,36,
246,nouadhibou,MR,20.9310,-17.0347,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:

# Set parameters for the hotel search
radius = 10000  # 10,000 meters
params = {
    "categories": "accommodation.hotel",
    "limit": 1,  # Only get the first result
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL for Geoapify Places API
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
alamos - nearest hotel: No hotel found
tamanrasset - nearest hotel: فندق الأمان
al ghayzah - nearest hotel: فندق تاج العرب
al bawiti - nearest hotel: Old Oasis Hotel
hoshangabad - nearest hotel: No hotel found
parauna - nearest hotel: No hotel found
al kharijah - nearest hotel: Kharga Hotel
khailar - nearest hotel: No hotel found
diplo - nearest hotel: No hotel found
nouadhibou - nearest hotel: Noadhibou Guesthouse
ouezzane - nearest hotel: No hotel found
san julian - nearest hotel: No hotel found
pinhao - nearest hotel: No hotel found
chui - nearest hotel: Alerces
metlili chaamba - nearest hotel: No hotel found
khanpur mahar - nearest hotel: No hotel found
tanout - nearest hotel: No hotel found
vacaria - nearest hotel: Pampa
sur - nearest hotel: Sur Hotel
ourinhos - nearest hotel: Hotel Premium Flat
utraula - nearest hotel: No hotel found
beinamar - nearest hotel: No hotel found
heroica caborca - nearest hotel: Hotel Posada del Desierto
ataq - nearest hotel: الشا

,City,Country,Lat,Lng,Humidity,Hotel Name
30,alamos,MX,27.0167,-108.9333,36,No hotel found
53,tamanrasset,DZ,22.7850,5.5228,16,فندق الأمان
71,al ghayzah,YE,16.2079,52.1760,55,فندق تاج العرب
92,al bawiti,EG,28.3492,28.8659,34,Old Oasis Hotel
97,hoshangabad,IN,22.7500,77.7167,50,No hotel found
147,parauna,BR,-16.9478,-50.4486,55,No hotel found
156,al kharijah,EG,25.4514,30.5464,30,Kharga Hotel
196,khailar,IN,25.3500,78.5333,37,No hotel found
240,diplo,PK,24.4667,69.5840,36,No hotel found
246,nouadhibou,MR,20.9310,-17.0347,49,Noadhibou Guesthouse


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng", "Lat",
    geo=True,
    size="Humidity",
    color="City",
    alpha=0.7,
    tiles="OSM",
    hover_cols=["Hotel Name", "Country"],
    title="Map of Cities with Nearby Hotels"
)

# Display the map
hotel_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)